## General prerprocess data for baseline models.

## Preprocess for both SARIMA and HWES, we need fixed time interval:

In [18]:
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import adfuller
from gluonts.dataset.common import ListDataset
import json
import os
import math

In [2]:
df = pd.read_parquet('../data/processed/cleaned.parquet')
price_cols_to_drop = [col for col in df.columns if col.startswith("price_") and col != "price_USD"]
df.drop(columns=price_cols_to_drop, inplace=True) #Delete all price rows except usd
df.head()
df.tail()

,mempool_blocks_blockSize,mempool_blocks_blockVSize,mempool_blocks_nTx,mempool_blocks_totalFees,mempool_blocks_medianFee,recommended_fee_fastestFee,recommended_fee_halfHourFee,recommended_fee_hourFee,recommended_fee_economyFee,recommended_fee_minimumFee,...,mempool_fee_histogram_bin_70_75,mempool_fee_histogram_bin_75_80,mempool_fee_histogram_bin_80_85,mempool_fee_histogram_bin_85_90,mempool_fee_histogram_bin_90_95,mempool_fee_histogram_bin_95_100,mempool_fee_histogram_bin_100_150,mempool_fee_histogram_bin_150_200,mempool_fee_histogram_bin_200_250,mempool_fee_histogram_bin_250_300
timestamp,,,,,,,,,,,,,,,,,,,,,
2025-04-15 09:34:42,1731329.0,997991.75,3810.0,4524017.0,2.149812,3.0,3.0,3.0,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2025-04-15 09:39:41,1593473.0,997958.75,3874.0,3061643.0,2.000000,2.0,2.0,2.0,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2025-04-15 09:44:41,1732510.0,997988.50,4511.0,4356829.0,1.554327,2.0,2.0,2.0,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2025-04-15 09:49:42,1768397.0,997952.00,4938.0,2320204.0,1.433566,2.0,2.0,2.0,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2025-04-15 09:54:42,1734405.0,997926.75,4746.0,3605544.0,2.000000,2.0,2.0,2.0,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Check if all the time interval are fixed.

In [3]:
time_diffs = df.index.to_series().diff()
print("Unique time intervals:")
print(time_diffs.value_counts())

most_common_interval = time_diffs.mode()[0]
print(f"\nMost common interval: {most_common_interval}")


Unique time intervals:
timestamp
0 days 00:05:00    4291
0 days 00:05:01    3555
0 days 00:04:59    3555
0 days 00:05:04     128
0 days 00:04:56     127
0 days 00:05:03     110
0 days 00:04:57     110
0 days 00:04:55       3
0 days 00:05:05       2
0 days 00:03:32       1
0 days 00:07:00       1
0 days 00:01:15       1
0 days 00:08:46       1
0 days 00:01:21       1
0 days 00:06:23       1
0 days 00:08:38       1
0 days 00:03:03       1
0 days 00:06:29       1
0 days 00:05:57       1
0 days 00:04:58       1
0 days 00:04:07       1
0 days 00:02:57       1
0 days 00:07:03       1
0 days 00:04:44       1
0 days 00:05:13       1
0 days 00:02:43       1
0 days 00:07:17       1
0 days 00:03:36       1
Name: count, dtype: int64

Most common interval: 0 days 00:05:00


In [4]:
# Resample to 5-minute regular intervals, averaging or interpolating values
# No actual missing data, just values not aligned exactly on the desired resampling grid, so we can interpolate the data.
df_resampled = df.resample("5min").mean()
# Linear interpolation is safe here due to small timing variations
df_resampled = df_resampled.interpolate(method='linear')
df_resampled.head()
df_resampled.tail()

,mempool_blocks_blockSize,mempool_blocks_blockVSize,mempool_blocks_nTx,mempool_blocks_totalFees,mempool_blocks_medianFee,recommended_fee_fastestFee,recommended_fee_halfHourFee,recommended_fee_hourFee,recommended_fee_economyFee,recommended_fee_minimumFee,...,mempool_fee_histogram_bin_70_75,mempool_fee_histogram_bin_75_80,mempool_fee_histogram_bin_80_85,mempool_fee_histogram_bin_85_90,mempool_fee_histogram_bin_90_95,mempool_fee_histogram_bin_95_100,mempool_fee_histogram_bin_100_150,mempool_fee_histogram_bin_150_200,mempool_fee_histogram_bin_200_250,mempool_fee_histogram_bin_250_300
timestamp,,,,,,,,,,,,,,,,,,,,,
2025-04-15 09:30:00,1731329.0,997991.75,3810.0,4524017.0,2.149812,3.0,3.0,3.0,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2025-04-15 09:35:00,1593473.0,997958.75,3874.0,3061643.0,2.000000,2.0,2.0,2.0,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2025-04-15 09:40:00,1732510.0,997988.50,4511.0,4356829.0,1.554327,2.0,2.0,2.0,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2025-04-15 09:45:00,1768397.0,997952.00,4938.0,2320204.0,1.433566,2.0,2.0,2.0,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2025-04-15 09:50:00,1734405.0,997926.75,4746.0,3605544.0,2.000000,2.0,2.0,2.0,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
#SARIMA need stationarity check
ts = df_resampled['recommended_fee_fastestFee']
adf_result = adfuller(ts.dropna())
print(f"ADF Statistic: {adf_result[0]}")
print(f"p-value: {adf_result[1]}")


ADF Statistic: -13.202001988695697
p-value: 1.0896816913188874e-24


The p-value is much less than 0.05, which strongly indicates the series is stationary.
This means: no differencing is needed before fitting ARIMA or SARIMA.

In [6]:
df_resampled.to_parquet("../data/processed/preprocessed_sarima_hwes.parquet")

## Preprocess for linear regression
Since we only want to find the relationship between the external features and response, we do not need lag here. For whole data, need to standard scale all the numeric columns, as well as log transform the response for better result, I will leave what it is here for now. Apply these transformation before fitting the model.

In [7]:
df_resampled.to_parquet("../data/processed/preprocessed_lr.parquet")

## Preprocess for XGboost

In [8]:
#We will need lagging feature for XGboost. Since each hour has 12 data points (60 ÷ 5), 48 hours = 576 lag steps.
def create_lag_features_fast(df, target_col, lags):
    lagged_dfs = [
        df[[target_col]].shift(lag).rename(columns={target_col: f'{target_col}_lag_{lag}'})
        for lag in lags
    ]
    return pd.concat([df] + lagged_dfs, axis=1)
lags = range(1, 577)  # 48 hours of 5-minute intervals
df_xgboost = create_lag_features_fast(df_resampled, 'recommended_fee_fastestFee', lags)
df_xgboost.head()

,mempool_blocks_blockSize,mempool_blocks_blockVSize,mempool_blocks_nTx,mempool_blocks_totalFees,mempool_blocks_medianFee,recommended_fee_fastestFee,recommended_fee_halfHourFee,recommended_fee_hourFee,recommended_fee_economyFee,recommended_fee_minimumFee,...,recommended_fee_fastestFee_lag_567,recommended_fee_fastestFee_lag_568,recommended_fee_fastestFee_lag_569,recommended_fee_fastestFee_lag_570,recommended_fee_fastestFee_lag_571,recommended_fee_fastestFee_lag_572,recommended_fee_fastestFee_lag_573,recommended_fee_fastestFee_lag_574,recommended_fee_fastestFee_lag_575,recommended_fee_fastestFee_lag_576
timestamp,,,,,,,,,,,,,,,,,,,,,
2025-03-05 02:10:00,2131827.0,997874.25,1030.0,2077770.0,1.300139,2.0,2.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-03-05 02:15:00,1986259.0,997882.00,1915.0,2663152.0,2.000000,2.0,2.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-03-05 02:20:00,1814202.0,997958.25,2710.0,3423846.0,2.413300,3.0,3.0,3.0,3.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-03-05 02:25:00,1604116.0,997937.50,3256.0,4136741.0,3.008581,4.0,3.0,3.0,3.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-03-05 02:30:00,2200654.0,997987.75,1015.0,1890732.0,1.362632,2.0,2.0,2.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
df_xgboost.to_parquet("../data/processed/preprocessed_xgboost.parquet")

## Advanced-deepar

In [10]:
df.index = pd.to_datetime(df.index) 
df_deepar = df
df_deepar.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 11901 entries, 2025-03-05 02:14:41 to 2025-04-15 09:54:42
Data columns (total 61 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   mempool_blocks_blockSize                  11900 non-null  float64
 1   mempool_blocks_blockVSize                 11900 non-null  float64
 2   mempool_blocks_nTx                        11900 non-null  float64
 3   mempool_blocks_totalFees                  11900 non-null  float64
 4   mempool_blocks_medianFee                  11900 non-null  float64
 5   recommended_fee_fastestFee                11901 non-null  float64
 6   recommended_fee_halfHourFee               11901 non-null  float64
 7   recommended_fee_hourFee                   11901 non-null  float64
 8   recommended_fee_economyFee                11901 non-null  float64
 9   recommended_fee_minimumFee                11901 non-null  float64
 10 

In [11]:
# Step 1: Extract timestamp and target
start_timestamp = df_deepar.index[0]  # assumes index is datetime
target_values = df_deepar["recommended_fee_fastestFee"].values.tolist()
start_timestamp

Timestamp('2025-03-05 02:14:41')

In [12]:
# Step 2: add dow, tod

# 1. Day of week (one-hot)
df_deepar['day_of_week'] = df.index.dayofweek
day_of_week_ohe = pd.get_dummies(df['day_of_week'], prefix='dow')

# 2. Time of day (cyclical features)
#We use sine/cosine pairs to represent cyclical (periodic) features like time of day, 
#day of week, or month of year, because raw integers fail to represent the circular relationship.
df_deepar['hour'] = df.index.hour
df_deepar['minute'] = df.index.minute
df_deepar['minute_of_day'] = df['hour'] * 60 + df['minute']
df_deepar['tod_sin'] = np.sin(2 * np.pi * df['minute_of_day'] / 1440)
df_deepar['tod_cos'] = np.cos(2 * np.pi * df['minute_of_day'] / 1440)

# 3. Merge all
df_deepar = pd.concat([df, day_of_week_ohe], axis=1)
df_deepar.head()

,mempool_blocks_blockSize,mempool_blocks_blockVSize,mempool_blocks_nTx,mempool_blocks_totalFees,mempool_blocks_medianFee,recommended_fee_fastestFee,recommended_fee_halfHourFee,recommended_fee_hourFee,recommended_fee_economyFee,recommended_fee_minimumFee,...,minute_of_day,tod_sin,tod_cos,dow_0,dow_1,dow_2,dow_3,dow_4,dow_5,dow_6
timestamp,,,,,,,,,,,,,,,,,,,,,
2025-03-05 02:14:41,2131827.0,997874.25,1030.0,2077770.0,1.300139,2.0,2.0,2.0,2.0,2.0,...,134,0.551937,0.833886,False,False,True,False,False,False,False
2025-03-05 02:19:42,1986259.0,997882.00,1915.0,2663152.0,2.000000,2.0,2.0,2.0,2.0,2.0,...,139,0.569997,0.821647,False,False,True,False,False,False,False
2025-03-05 02:24:41,1814202.0,997958.25,2710.0,3423846.0,2.413300,3.0,3.0,3.0,3.0,2.0,...,144,0.587785,0.809017,False,False,True,False,False,False,False
2025-03-05 02:29:42,1604116.0,997937.50,3256.0,4136741.0,3.008581,4.0,3.0,3.0,3.0,2.0,...,149,0.605294,0.796002,False,False,True,False,False,False,False
2025-03-05 02:34:42,2200654.0,997987.75,1015.0,1890732.0,1.362632,2.0,2.0,2.0,2.0,2.0,...,154,0.622515,0.782608,False,False,True,False,False,False,False


In [13]:
#Step3: change all fee from wide to long
fee_columns = [
    'recommended_fee_fastestFee',
    'recommended_fee_halfHourFee',
    'recommended_fee_hourFee',
    'recommended_fee_economyFee',
    'recommended_fee_minimumFee'
]
df_long = df_deepar[fee_columns].copy()
df_long["timestamp"] = df.index  # restore timestamp if it's the index

# Melt to long format
df_long = df_long.melt(
    id_vars=["timestamp"],
    value_vars=fee_columns,
    var_name="fee_type",
    value_name="fee_value"
)
df_merged = pd.merge(df_long, df_deepar, on="timestamp", how="left")
df_merged_sorted = df_merged.sort_values(by=["fee_type", "timestamp"]).reset_index(drop=True)
df_merged_sorted.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59505 entries, 0 to 59504
Data columns (total 77 columns):
 #   Column                                    Non-Null Count  Dtype         
---  ------                                    --------------  -----         
 0   timestamp                                 59505 non-null  datetime64[ns]
 1   fee_type                                  59505 non-null  object        
 2   fee_value                                 59505 non-null  float64       
 3   mempool_blocks_blockSize                  59500 non-null  float64       
 4   mempool_blocks_blockVSize                 59500 non-null  float64       
 5   mempool_blocks_nTx                        59500 non-null  float64       
 6   mempool_blocks_totalFees                  59500 non-null  float64       
 7   mempool_blocks_medianFee                  59500 non-null  float64       
 8   recommended_fee_fastestFee                59505 non-null  float64       
 9   recommended_fee_halfHourFee 

In [14]:
#Step 4: Create list dataset for deepar
print(type(df_merged_sorted["timestamp"].iloc[0])) 
fee_type_to_id = {ft: i for i, ft in enumerate(df_merged_sorted["fee_type"].unique())}
external_cols = df_merged_sorted.columns.difference(["timestamp", "fee_type", "fee_value"])

series_list = []

for fee_type, group in df_merged_sorted.groupby("fee_type"):
    group = group.sort_values("timestamp")

    series = {
        "start": group["timestamp"].iloc[0].strftime("%Y-%m-%d %H:%M:%S"),
        "target": group["fee_value"].tolist(),
        "feat_dynamic_real": group[external_cols].astype(float).T.values.tolist(),
        "feat_static_cat": [int(fee_type_to_id[fee_type])]
    }

    series_list.append(series)



<class 'pandas._libs.tslibs.timestamps.Timestamp'>


In [21]:
feat_dynamic_real_names = list(external_cols)

def sanitize_for_json(obj):
    """Recursively replace NaN with None so JSON can parse it."""
    if isinstance(obj, float) and math.isnan(obj):
        return None
    elif isinstance(obj, list):
        return [sanitize_for_json(x) for x in obj]
    elif isinstance(obj, dict):
        return {k: sanitize_for_json(v) for k, v in obj.items()}
    else:
        return obj
with open("../data/processed/deepar_dataset.jsonl", "w") as f:
    for item in series_list:
        clean_item = sanitize_for_json(item)
        json.dump(clean_item, f)
        f.write("\n")

# Save mapping
with open("../data/processed/fee_type_to_id.json", "w") as f:
    json.dump(fee_type_to_id, f)

with open("../data/processed/feat_dynamic_real_names.json", "w") as f:
    json.dump(feat_dynamic_real_names, f, indent=2)
